## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-01-12-00 +0000,wsj,FAA Ramps Up Probe on Airlines’ Flight Reducti...,https://www.wsj.com/business/airlines/faa-ramp...
1,2025-12-02-01-11-18 +0000,wapo,What we know about the National Guard shooting...,https://www.washingtonpost.com/national-securi...
2,2025-12-02-01-10-58 +0000,nyt,Trump Administration Revokes Licenses of Thous...,https://www.nytimes.com/2025/12/01/us/politics...
3,2025-12-02-01-07-22 +0000,bbc,Why more young US women appear ready to move a...,https://www.bbc.com/news/articles/cgqz19ede19o...
4,2025-12-02-01-04-12 +0000,missionlocal,Waymo confirms its car hit dog in Western Addi...,https://missionlocal.org/2025/12/waymo-confirm...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,46
103,house,20
101,new,16
102,white,14
244,drug,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,119
98,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...,114
198,2025-12-01-14-38-59 +0000,nypost,Trump backs Hegseth after report claims war se...,https://nypost.com/2025/12/01/us-news/trump-ba...,108
116,2025-12-01-20-17-00 +0000,wsj,The White House said Trump underwent advanced ...,https://www.wsj.com/politics/white-house-says-...,105
194,2025-12-01-14-46-30 +0000,nypost,What a difference an admin makes as first lady...,https://nypost.com/2025/12/01/us-news/first-la...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,119,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...
198,62,2025-12-01-14-38-59 +0000,nypost,Trump backs Hegseth after report claims war se...,https://nypost.com/2025/12/01/us-news/trump-ba...
196,47,2025-12-01-14-42-17 +0000,nypost,Appeals court disqualifies former Trump lawyer...,https://nypost.com/2025/12/01/us-news/appeals-...
51,44,2025-12-01-23-10-17 +0000,nypost,Rapper targeted in Stockton mass shooting is h...,https://nypost.com/2025/12/01/us-news/rapper-t...
218,40,2025-12-01-12-20-44 +0000,nypost,Hong Kong arrests more suspects in fire probe ...,https://nypost.com/2025/12/01/world-news/hong-...
47,38,2025-12-01-23-21-47 +0000,nypost,Son of drug kingpin ‘El Chapo’ pleads guilty i...,https://nypost.com/2025/12/01/us-news/son-of-d...
141,37,2025-12-01-18-53-03 +0000,nypost,Starbucks to pay $35M to NYC workers to settle...,https://nypost.com/2025/12/01/business/starbuc...
111,32,2025-12-01-20-31-23 +0000,nypost,"Bannon, top conservatives urge White House to ...",https://nypost.com/2025/12/01/business/bannon-...
277,31,2025-12-01-04-00-00 +0000,wsj,Economists predicted a global shock from Presi...,https://www.wsj.com/economy/trade/americas-tar...
214,31,2025-12-01-12-59-45 +0000,nypost,Venezuela’s Maduro spotted for first time in d...,https://nypost.com/2025/12/01/us-news/venezuel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
